In [2]:
import os
from dotenv import load_dotenv
from sqlalchemy.exc import SQLAlchemyError

# .env 파일 로드
load_dotenv()

# 환경 변수 사용
db_host = os.getenv("DATABASE_HOST")
db_name = os.getenv("DATABASE_NAME")
db_user = os.getenv("DATABASE_USER")
db_password = os.getenv("DATABASE_PASSWORD")

In [3]:
import random

class RandomNicknameGenerator:
    # 형용사 리스트
    adjectives = [
        "배부른", "행복한", "용감한", "활기찬", "지혜로운",
        "즐거운", "슬기로운", "긍정적인", "평화로운", "창의적인"
    ]

    # 명사 리스트
    nouns = [
        "토끼", "사자", "호랑이", "고양이", "강아지",
        "여우", "나무늘보", "팬더", "코알라", "펭귄",
        "햄스터", "다람쥐", "앵무새", "알파카", "쿼카"
    ]

    @staticmethod
    def generate_nickname():
        # 형용사와 명사 리스트에서 랜덤으로 하나씩 선택
        adjective = random.choice(RandomNicknameGenerator.adjectives)
        noun = random.choice(RandomNicknameGenerator.nouns)

        # 0에서 9999 사이의 랜덤 숫자 생성
        number = random.randint(0, 9999)

        # 닉네임 조합
        return f"{adjective} {noun} {number}"

print(RandomNicknameGenerator.generate_nickname())

지혜로운 토끼 7560


In [4]:
import psycopg2

connection = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password,
    port="5432"  # 기본 포트
)

print (connection)
cursor = connection.cursor()

# 데이터베이스 작업 수행
cursor.execute("SELECT * FROM \"CulturePost\";")
results = cursor.fetchall()
for row in results:
    print(row)

cursor.close()
connection.close()

<connection object at 0x10562f6d0; dsn: 'user=postgres password=xxx dbname=reculture host=35.226.243.3 port=5432', closed: 0>
(1, '파일럿', '✈️', datetime.datetime(2024, 8, 20, 16, 13, 53, 726000), 'PUBLIC', '파일럿', '조정식, 이주명, 한선화 출연 / 김한결 감독', '코미디', '', 1, 1, '역시 믿고 보는 코미디 영화!', datetime.datetime(2024, 8, 13, 15, 9, 10, 606000), datetime.datetime(2024, 8, 20, 16, 14, 55, 735000))
(4, '8/21 movie', '\U0001f979', datetime.datetime(2024, 8, 20, 17, 18, 56), 'PRIVATE', 'Troll', 'Idk', 'Animation', '', 2, 1, 'Cool', datetime.datetime(2024, 8, 20, 17, 20, 23, 393000), datetime.datetime(2024, 8, 20, 17, 20, 23, 393000))
(5, '8/15 musical', '🥲', datetime.datetime(2024, 8, 15, 5, 23, 53), 'PRIVATE', '영웅', 'somewhere', 'who', '', 2, 2, 'nice', datetime.datetime(2024, 8, 21, 5, 25, 1, 495000), datetime.datetime(2024, 8, 21, 5, 25, 1, 495000))
(6, '8/15 Book', '😗', datetime.datetime(2024, 8, 15, 5, 25, 38), 'PUBLIC', 'Book', 'Me', '7.15~8/15', 'Gg', 2, 7, 'Ggg', datetime.datetime(2024, 8, 21, 5, 26,

In [42]:
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy import Column, Integer, String, ForeignKey, DateTime
from datetime import datetime

class Base(DeclarativeBase):
    pass
class User(Base):
    __tablename__ = "User"
    id = Column(Integer, primary_key=True)
    email = Column(String, unique=True)
    password = Column(String)
    
    def __repr__(self) -> str:
        return f"<User {self.id} {self.email}>"

class Profile(Base):
    __tablename__ = "Profile"
    id = Column(Integer, primary_key=True)
    userId = Column(Integer, ForeignKey("User.id"), nullable=False)
    nickname = Column(String, nullable=False)
    profilePhoto = Column(String, nullable=False)
    bio = Column(String, nullable=True)
    birthdate = Column(DateTime, nullable=True)
    
    def __repr__(self) -> str:
        return f"<Profile {self.id} {self.nickname} {self.userId}>"

class CulturePost(Base):
    __tablename__ = "CulturePost"
    id = Column(Integer, primary_key=True)
    title = Column(String, nullable=False)
    emoji = Column(String, nullable=False)
    date = Column(DateTime, nullable=False)
    categoryId = Column(Integer, nullable=False)
    authorId = Column(Integer, ForeignKey("User.id"), nullable=False)
    review = Column(String, nullable=False)
    disclosure = Column (String, nullable=False)
    detail1 = Column (String, nullable=True)
    detail2 = Column (String, nullable=True)
    detail3 = Column (String, nullable=True)
    detail4 = Column (String, nullable=True)
    createdAt = Column(DateTime, insert_default=datetime.now())
    updatedAt = Column(DateTime, default=datetime.now())
    
    def __repr__(self) -> str:
        return f"<CulturePost {self.id} {self.title} {self.emoji} {self.review}>"
    
class Photo(Base):
    __tablename__ = "Photo"
    id = Column(Integer, primary_key=True)
    url = Column(String, nullable=False)
    culturePostId = Column(Integer, ForeignKey("CulturePost.id"), nullable=False)
    
    def __repr__(self) -> str:
        return f"<Photo {self.url} {self.culturePostId}>"

In [24]:
from sqlalchemy import create_engine, MetaData, Column
from sqlalchemy.orm import sessionmaker

# 데이터베이스 연결 설정
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}')

# 메타데이터 객체 생성
metadata = MetaData()

# 세션 생성기 설정
Session = sessionmaker(bind=engine)
session = Session()

# ORM을 사용하여 테이블에 액세스
# 예시: session.query(User).all()

users = session.query(User).all()
for user in users:
    print (user)
    
profiles = session.query(Profile).all()
for profile in profiles:
    print (profile)
    
posts = session.query(CulturePost).all()
for post in posts:
    print (post)
    
photos = session.query(Photo).all()
for photo in photos:
    print (photo)

<User 2 syhhwang1231@gmail.com>
<User 3 yjkwon424@naver.com>
<User 4 suyeon@test.com>
<User 6 suyeon2@test.com>
<User 8 123@test.com>
<User 11 suyeon1@test.com>
<User 14 yjkwon424@gmail.com>
<User 15 Jini00@naver.com>
<User 16 Fff@gmail.com>
<User 18 Oxo@gmail.com>
<User 19 suyeon0903@test.com>
<User 21 suyeon09031@test.com>
<User 22 suyeon09032@test.com>
<User 23 jini@naver.com>
<User 24 suyeon09033@test.com>
<User 25 suyeon09034@test.com>
<User 26 suyeon09035@test.com>
<User 27 suyeon0904@test.com>
<User 28 suyeon09042@test.com>
<User 29 suyeon09043@test.com>
<User 31 suyeon09044@test.com>
<User 32 suyeon09045@test.com>
<User 33 suyeon09046@test.com>
<User 34 >
<User 35 suyeonnn@test.com>
<User 36 Suyeonnn@test.com>
<User 1 soomin@gmail.com>
<User 37 Star@naver.com>
<User 42 pythontest@email.com>
<Profile 1 soomin 1>
<Profile 3 지니00 15>
<Profile 4 oxo 18>
<Profile 5 Jini 23>
<Profile 6 suyeonn 36>
<Profile 7 별별 37>
<Profile 2 suyeon 2>
<Profile 8 python-test 42>
<CulturePost 1 파일럿 ✈️

In [27]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}')

Session = sessionmaker(bind=engine)
session = Session()

try:
    new_user = User(email=f"pythontest2@email.com", password="12345678")
    session.add(new_user)
    session.flush()
    print(new_user)
    
    new_profile = Profile(userId=new_user.id, nickname=RandomNicknameGenerator.generate_nickname(), profilePhoto="/uploads/profile/default.png")
    session.add(new_profile)
    
    session.commit()
    print(new_profile)
except SQLAlchemyError as e:
    session.rollback()
    print(f"Error: {e}")
finally:
    session.close()

<User 43 pythontest2@email.com>
<Profile 9 창의적인 사자 1027 43>


In [31]:
# 드라마 유저 넣기
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from sqlalchemy.sql import func
from sqlalchemy.exc import SQLAlchemyError

engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}')

Session = sessionmaker(bind=engine)
session = Session()

for x in range(300):
    try:
        new_user = User(email=f"drama{x}@gmail.com", password="12345678")
        session.add(new_user)
        session.flush()
        print(new_user)
    
        nickname = RandomNicknameGenerator.generate_nickname()
        new_profile = Profile(userId=new_user.id, nickname=nickname, bio=nickname, profilePhoto="/uploads/profile/default.png", birthdate=func.now())
        session.add(new_profile)
    
        session.commit()
        print(new_profile)
    except SQLAlchemyError as e:
        session.rollback()
        print(f"Error: {e}")

session.close()

NameError: name 'SQLAlchemyError' is not defined

In [35]:
import random
from datetime import datetime, timedelta

def random_date(start, end):
    delta = end - start
    random_days = random.randint(0, delta.days)
    return start + timedelta(days=random_days)

# 예시: 2023년 1월 1일 ~ 2023년 12월 31일 사이의 랜덤 날짜 생성
start_date = datetime(2024, 6, 1)
end_date = datetime(2024, 9, 14)

random_date_value = random_date(start_date, end_date)
print(random_date_value)  # 임의의 날짜 출력

2024-08-04 00:00:00


In [46]:
# 드라마 기록 넣기
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd
import random

engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}')

Session = sessionmaker(bind=engine)
session = Session()

csv_file = "drama_reviews.csv"
data = pd.read_csv(csv_file)

for index, row in data.iterrows():
    try:
        genre = ""
        if row['drama_id'] == 79978:
            genre = "드라마, 멜로/로맨스, 판타지, 코미디"
        elif row['drama_id'] == 80083:
            genre = "드라마, 코미디, 음악"
        elif row['drama_id'] == 130294:
            genre = "멜로/로맨스, 판타지"
        elif row['drama_id'] == 130737:
            genre = "드라마"
        
        dt = random_date(start_date, end_date)
        dt2 = random_date(dt, end_date)
        new_post = CulturePost(
            title = row['drama_name'],
            emoji = "📺",
            date = dt,
            categoryId = 6,
            authorId = random.randint(44, 344),
            review = row['review_title'],
            disclosure = "PUBLIC",
            detail1 = row['drama_name'],
            detail2 = genre,
            createdAt= dt2
        )
        session.add(new_post)
        session.flush()
        print (new_post)
        
        new_photo = Photo(
            url = f"/uploads/drama/drama_{row['drama_id']}.webp",
            culturePostId = new_post.id,
        )
        session.add(new_photo)
        session.commit()
        print(new_photo)
    except SQLAlchemyError as e:
        session.rollback()
        print(f"Error: {e}")

session.close()

<CulturePost 558 선재 업고 튀어 📺 우리가 찾던 바로 그 청춘 로맨스물. 안 본 사람은 있어도 한번만 본 사람은 없는, 늦지않게 정주행해야 하는 작품. 연출진의 계산된 연출과 각본이 유치하지 않은 설렘으로 다가온다.>
<Photo /uploads/drama/drama_130294.webp 558>
<CulturePost 559 선재 업고 튀어 📺 쫀쫀한 스토리에 숨겨진 떡밥, 복선을 찾아가는 재미가 쏠쏠하다.
김혜윤 배우의 흡입력있는 연기력과 변우석 배우의 91년생임이 믿기지 않는 청춘 비주얼, 아름다운 색감까지 묘미이다>
<Photo /uploads/drama/drama_130294.webp 559>
<CulturePost 560 선재 업고 튀어 📺 어?왜 이거 재밌어 ㅋㅋ>
<Photo /uploads/drama/drama_130294.webp 560>
<CulturePost 561 선재 업고 튀어 📺 요즘 선재업고 튀어에 빠져있습니다. 그 이유는>
<Photo /uploads/drama/drama_130294.webp 561>
<CulturePost 562 선재 업고 튀어 📺 1화 눈물 콧물 ㅠㅠ 두배우 키차이 ... 👍  
김혜윤배우의 청량함과 변우석배우의 잡아주는 연기가 드라마와 잘어울려서 기분좋아진다>
<Photo /uploads/drama/drama_130294.webp 562>
<CulturePost 563 선재 업고 튀어 📺 초여름이 될 때 마다 생각나는 인생 청춘 드라마!!  김혜윤 배우의 풋풋함과 뽀쟉함만으로도 너무 재밌다. 16부작임에도 1년 동안 본 것 같은 탄탄한 서사가 있는 드라마는 처음!>
<Photo /uploads/drama/drama_130294.webp 563>
<CulturePost 564 선재 업고 튀어 📺 누군가의 팬이라면 설렐만한 드라마!
원작과 내용이 다르지만 드라마만의 매력이 가득 담겨 있어요>
<Photo /uploads/drama/drama_130294.webp 564>
<CulturePost 5